In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement

import pandas as pd
from pathlib import Path

from mbp.webscraping import activate_web_driver, get_team_page, get_game_stats
from mbp.paths import RAW_DATA_DIR, SEASONS_DIR, team_save_dir
from mbp.data import download_game_data

In [3]:
team_name = "Arizona"
year = 2022
opponent_name = "Southern U."

In [1]:
(team1, stats1, team2, stats2) = download_game_data(team_name, opponent_name, year)
print(f"{team1} vs. {team2}")

NameError: name 'download_game_data' is not defined

In [ ]:
### Everything below is scratch

In [218]:
driver = activate_web_driver("firefox", True)
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)

(team1, stats1, team2, stats2) = get_game_stats(driver, df, team_name, opponent_name, year)
print(team1, team2)

Southern U. Arizona


95-78
['3fe9931c-86e6-486c-9d88-a4bb1218272d']
0 []


IndexError: list index out of range

In [216]:
l = [1,2,3,4]
l[-2:]

[3, 4]

In [154]:
driver = activate_web_driver("firefox", False)
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)

get_team_page(driver, df, team_name, year)
# Get game page
game_by_game_link = driver.find_element(By.LINK_TEXT, "Game By Game")
game_by_game_link.click()
wait = WebDriverWait(driver, 10)
wait.until(lambda d: driver.find_element(By.CSS_SELECTOR, "#game_breakdown_div table"))

# Save the main window handle
curr_window_handle = driver.current_window_handle

rows = driver.find_elements(By.CSS_SELECTOR, "#game_breakdown_div table > tbody tr")
# rows = opp_table.find_elements(By.TAG_NAME, "tr")
# skip the first two rows
rows = [row for row in rows[2:] if row.get_attribute("class") == ""]
for idx, row in enumerate(rows):
    try:
        cells = rows[idx].find_elements(By.TAG_NAME, "td")
    except:
        pass
    if cells[1].text.strip() == opponent_name:
        row_links = rows[idx].find_elements(By.TAG_NAME, "a")
        print(row_links[2].text)
        row_links[2].click()
        break

# Wait for the new window or tab
wait.until(EC.number_of_windows_to_be(2))

# Switch to new tab
other_handles = [handle for handle in driver.window_handles if handle != curr_window_handle]
print(other_handles)
if len(other_handles) > 0:
    driver.switch_to.window(other_handles[0])
else:
    print("Something went wrong")

95-78
['0529f7b3-c170-4b9f-8291-4de39220d6b7']


In [194]:
# Get the last two tables
tables = driver.find_elements(By.CSS_SELECTOR, "table.mytable")
tables

def process_table(table: WebElement):
    # First table heading
    first_table_heading = table.find_element(By.CSS_SELECTOR, "tbody > tr.heading")
    team = first_table_heading.text
    print(team)

    # Get all headers
    headers = table.find_elements(By.CSS_SELECTOR, "th")
    all_headings = [heading.text for heading in headers]
    print(all_headings)

    # get all players for each row
    player_rows = table.find_elements(By.CSS_SELECTOR, "tr.smtext")
    players_and_stats = []
    for row in player_rows:
        player_and_stats = {}
        cells = row.find_elements(By.TAG_NAME, "td")
        for idx, heading in enumerate(all_headings):
            player_and_stats[heading] = cells[idx].text
        
        players_and_stats.append(player_and_stats)

    game_df = pd.DataFrame(players_and_stats)
    return (team, game_df)


team1, stats_team1 = process_table(tables[1])
team2, stats_team2 = process_table(tables[2])

Southern U.
['Player', 'Pos', 'G', 'MP', 'FGM', 'FGA', '3FG', '3FGA', 'FT', 'FTA', 'PTS', 'ORebs', 'DRebs', 'Tot Reb', 'AST', 'TO', 'STL', 'BLK', 'Fouls', 'DQ', 'Tech Fouls']
Arizona
['Player', 'Pos', 'G', 'MP', 'FGM', 'FGA', '3FG', '3FGA', 'FT', 'FTA', 'PTS', 'ORebs', 'DRebs', 'Tot Reb', 'AST', 'TO', 'STL', 'BLK', 'Fouls', 'DQ', 'Tech Fouls']


In [195]:
team1_games_save_dir = team_save_dir(team1, year) / "games"
if not team1_games_save_dir.exists():
    team1_games_save_dir.mkdir(exist_ok=True, parents=True)
stats_team1.to_csv(team1_games_save_dir / f"{team2}.csv")

team2_games_save_dir = team_save_dir(team2, year) / "games"
if not team2_games_save_dir.exists():
    team2_games_save_dir.mkdir(exist_ok=True, parents=True)
stats_team1.to_csv(team2_games_save_dir / f"{team1}.csv")